<a href="https://colab.research.google.com/github/zwang86/DLSys-Fall-2022-Report/blob/main/SparceArray_Graph_Networks_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

We extend the needle library by adding sparse matrices and related operations and autodiff. It supports both CPU and CUDA backends. On top of that, we build the Graph Convolutional Network and the Graph Attention Network. We train and compare these two networks and a baseline fully-connected network on the Cora dataset, and show that graph networks can achieve higher accuracy without too much performance cost.

# Environment Setup



The project is available in [Github repository](https://github.com/MashPlant/dlsys-proj). You may need to request access first.
You can also access the code directly from [shared directory](https://drive.google.com/drive/folders/1bu0THsbS5IuQzBHvFPdmy6qdbcuNTAS7?usp=share_link). 

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
!mkdir -p 10714
%cd /content/drive/MyDrive/10714

In [ ]:
# Configure Github and repository
!git config --global user.email "{YOUR_EMAIL}"
!git config --global user.name "{YOUR_USERNAME}"
!git clone https://"{GITHUB_TOKEN}"@github.com/MashPlant/dlsys-proj.git
%cd /content/drive/MyDrive/10714/dlsys-proj
!git pull

In [ ]:
# Add Python PATH
import sys
sys.path.append('./python')

In [ ]:
# Install pybind11
!pip3 install pybind11

In [ ]:
# Build CPU and CUDA backend
!make

-- Found pybind11: /usr/local/lib/python3.8/dist-packages/pybind11/include (found version "2.10.1")
-- Found cuda, building cuda backend
Sun Dec  4 20:48:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    29W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+------------------------------

# Sparse Matrix Operations

## Construction and Conversion

For simplicity, we only support 2-D sparse matrices instead of general multidimensional sparse tensors, considering their usage in the project. There are many data formats to represent sparse matrices, such as DOK, COO, LIL, CSR, etc. We choose to use the CSR format. The Compressed Sparse Row (CSR) format is a common way of storing sparse matrices in memory. A matrix is represented as three arrays:

- `row`: Length is the number of rows plus one. It marks the starting and ending positions of a row in the `col` and `val` array. Specifically, the elements in row `r` are stored in `col[row[r]..row[r + 1]]` and `val[row[r]..row[r + 1]]` (left-closed and right-open interval).
- `col`: Length is the number of non-zero elements. It contains the column indices of the non-zero elements.
- `val`: Length is the number of non-zero elements. It contains the values of the non-zero elements.

It has some advantages that we value, including less memory usage (only 8 bytes for each `float32` non-zero element) and a hierarchical structure that can enable parallelism (especially for CUDA in the project).

We do not modify the `NDArray` class from the original `needle` library. Instead, we create a `SparseArray` class with a similar status with `NDArray`, which means that `Value.cached_data` can be either a `NDArray` or a `SparseArray`.
We export a set of functions with `sparse` as the name prefix in CPU and CUDA backends. They are used in the methods in `SparseArray`, and are mostly straightforward.

We provide two methods to convert a sparse matrix from/to a dense array, which is represented as a contiguous piece of memory.

- `sparse_from_dense`: Convert a dense array to a `SparceArray` instance. It iterates over all the elements and only keeps the non-zero ones.
- `sparse_to_dense`: Convert a `SparseArray` instance to a dense array. It zeros the ouput array and fill the non-zero elements.

## Operations

We also define a similar set of methods in `SparseArray` as `NDArray`. Since the operations in `ops.py` mostly just forward the call to the underlying array class, we can reuse most of them, and only need to do very few modifications.

There is a slight difference in implementation. The operations of `NDArray` normally first allocates an `NDArray` instance, and then stores the calculation result in it. On the other hand, the number of non-zero elements of the resulting `SparseArray` may depend on the specific operation, and we cannot allocate it in advance, so we let these functions return `SparseArray` directly.

In order to support both CPU and CUDA backends, for most operations, we can simply replace the outermost one or two loops of the CPU version with CUDA blocks or block+threads.
For some operations (such as `sparse_gat_grad`), doing this directly will cause data racing, and we need to modify the computation order to adapt to GPU.
For some operations (such as `sparse_transpose`), it is too hard to implement them with GPU, and we choose to transfer the data to CPU, invoke the CPU version, and transfer the data to GPU.

- `sparse_[add|sub|mul|div|rdiv]_[scalar|dense|sparse]` (in total 15 functions): Perform basic scalar or elemen-wise operations on sparse-scalar, sparse-dense and sparse-sparse operands. For sparse-sparse, we only support the case where the non-zero element locations of the two operands are the same. This is enough for our usage.
- `sparse_dense_matmul`: Perform matrix multiplication on a sparse matrix and a dense matrix. Suppose the two operands have shape `m x n` and `n x k` respectively, then the dense-dense matmul takes `O(mnk)` time, while the sparse-dense matmul takes `O((number of non-zero elements in A) x k)` time. Since the edge matrix is sparse in our usage, it is very likely to reduce the cubic time into quadratic time.
- `sparse_transpose`: The transposition of CSR matrices is a little tricky to implement efficiently. By definition, the `row` array in CSR maintains an exclusive prefix sum of the number of elements in each row. We first count the number of elements in each column (because of transposition), and then compute an inclusive prefix sum of the counters. We then visits elements in the original matrix. By using this inclusive prefix sum and modifying it in the fly, we can place the elements correctly and get the exclusive prefix sum. We think that this is too complex for a GPU implementation.  
- `sparse_gat`: We will elaborate this in the GAT section.
- `sparse_gat_grad_from_[sparse|dense]`: The gradient of the `sparse_gat`. It takes the upstream gradient and calculates the operands' final gradients directly (i.e, the caller doesn't need to do the chain-rule multiplication). The `[sparse|dense]` variants are explained in the following section.
- `sparse_softmax`: The softmax function for sparse matrices on axis-1. 
- `sparse_softmax_grad_from_[sparse|dense]`: The gradient of `sparse_softmax`, similar to `sparse_gat_grad_from_[sparse|dense]`.

## Autograd

It is mostly naturally supported because of the reuse in `ops.py`. However, there is an important difference in `sparse_gat` and `sparse_softmax`. Ideally, we should implement a complex operation by decomposing it into several differentiable simple operations. However, we feel it too complex to do so for them. Instead, we implement the computation and gradient logic directly in the backend. Doing so disables us from taking the second-order derivative even though the operation is second-order differentiable, but this has no impact on our usage. Anouther resulting problem is that the upstream gradient matrix may be dense or sparse. We could convert them to one format and handle only that format, but we think it has a performance overhead. So we implement the `[sparse|dense]` variants (with some C++ template tricks, this hardly increases the amount of code).

## Modules

For the modules, we add a few new modules to support the DNN we want to build in this project, including `GraphConvolution`, `GraphAttention`, and `MultiHeadGAT`. Plus 20 modules we implemented in the previous homework, the total of 23 modules have now supported both dense and sparse matrices and be able to make some powerful DNNs.


## Tests

Note that each test includes many shapes. We did not make `shape` a parameter of testing like our homework, as it would produce too many tests.

In [ ]:
# A few test on our new ops with cpu/cuda backend 
!python3 -m pytest -l -v -k "test_sparse"

============================= test session starts ==============================
platform linux -- Python 3.8.15, pytest-3.6.4, py-1.11.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/10714/dlsys-proj, inifile:
plugins: typeguard-2.7.1
collected 24 items                                                             

tests/test_sparse.py::test_sparse_init[device0] PASSED                   [  4%]
tests/test_sparse.py::test_sparse_init[device1] PASSED                   [  8%]
tests/test_sparse.py::test_sparse_matmul_forward[device0] PASSED         [ 12%]
tests/test_sparse.py::test_sparse_matmul_forward[device1] PASSED         [ 16%]
tests/test_sparse.py::test_sparse_matmul_backward[device0] PASSED        [ 20%]
tests/test_sparse.py::test_sparse_matmul_backward[device1] PASSED        [ 25%]
tests/test_sparse.py::test_sparse_add[device0] PASSED                    [ 29%]
tests/test_sparse.py::test_sparse_add[device1] PASSED                    [ 33%]

# Graph Convolutional Networks

Our implementation is based on the paper "[Simplifying Graph Convolutional Networks](https://arxiv.org/abs/1902.07153)" published in 2019 (although we didn't do the "simplifying" part, we find its illustration for the original GCNs easy to follow).

Graph Convolutional Networks (GCNs) are a type of neural network that can process data represented as a graph. They are a generalization of convolutional networks, which are commonly used for image processing. Like convolutional networks, GCNs use convolutional layers to extract local features from the graph structure. The locality in the figure is reflected in the neighborhood relationship within certain hops. As we will describe, it is actually done may matrix multiplication instead of the convolution operator.

The GCN operates on a graph $G$ with adjacency matrix $E$ and a set of $n$ nodes $V$. $A$ is a sparse matrix where $e_{ij}$ denotes the edge weight between nodes $v_i$ and $v_j$ (or simply 0/1 to denote whether an edge exists). The spirit of a graph convolution layer is to pass the information of a node's neighbors to the node. Suppose the input of a layer is $H^{(k-1)} \in \mathbf{R}^{n,f}$, where there is a feature vector of length $f$ associated with each node. We compute $H^{(k)} = \sigma(E H^{(k-1)} W)$ as the output of the layer. First, $H^{(k-1)} W$ can be considered as a linear preprocessing layer (can also be extended with a bias term). And then, we multiply it with $A$, which means that the new feature vector of a node will be a (weighted) sum of all the transformed feature vectors of its neighbors. At last, we apply an activation function $\sigma$.

```

in_features -> self.preprocess -> hidden_features * num_heads -> self.gn_layers -> hidden_features * num_heads -> self.postprocess -> out_features

where:
  1. self.preprocess is a nn.Linear layer with in_features and hidden_features * num_heads as input and output, respectively
  2. self.gn_layers is a list of nn.MultiHeadGAT layers with hidden_features * num_heads as input and hidden_features * num_heads as output
  3. self.postprocess is a nn.Linear layer with hidden_features * num_heads as input and out_features as output
```

# Graph Attention Networks

Our implementation is based on the paper "[Graph Attention Networks](https://arxiv.org/abs/1710.10903)" published in 2017. The paper introduces attention mechanisms to the graph structure. Graph Attention Networks (GATs) enhance the original GCNs by allowing the network to learn the importance of different nodes and edges and focus on the most relevant parts when making predictions. Recall that in the original GCNs described above, all neighborhoods contribute to a node with fixed weights (in the case where there are no prescribed edge weights, it is actually a fixed and equal weight), which may not be desirable because they are inherently different.

We will use the same graph notation (adjacency matrix $E$ and nodes $V$) in GAT as in GCN. The attention coefficients in a GAT layer are computed using the following formula:

$$b_{ij} = f (H_i, H_j)$$

$b_{ij}$ is only calucalted if node $j$ is a neighbor of node $i$ (i.e, $e_{ij} \ne 0$), so it is also sparse. It indicates the importance of node $j$'s features to node $i$. $f$ is a $\mathbf{R}^{f} \times \mathbf{R}^{f} \rightarrow \mathbf{R}$ function, and $H_i$ and $H_j$ are the node features of nodes $i$ and $j$ in the input features. The paper suggests that $f$ can be a single-layer feedforward neural network, which means that we maintain a learnable weight vector $\vec{a} \in \mathbf{R}^{2f}$, and $f (H_i, H_j) = \sigma(\vec{a}^T (H_i \parallel H_j))$ ($\parallel$ denotes concatenation). The activation function $\sigma$ is further picked as $\text{LeakyReLU}$.

The coefficients $b_{ij}$ are then used to compute the attention weights:

$$a_{ij} = \text{softmax}_j(b_{ij}) = \frac{\exp(b_{ij})}{\sum_{k, e_{ik} \ne 0} \exp(b_{ik})} = \frac{\exp(\text{LeakyReLU}(\vec{a}^T (H_i \parallel H_j)))}{\sum_{k, e_{ik} \ne 0} \exp(\text{LeakyReLU}(\vec{a}^T (H_i \parallel H_k)))}$$

<p align="center">
  <img src="https://raw.github.com/zwang86/DLSys-Fall-2022-Report/main/img/attention_weights.png" alt="Attention Weights"/>
</p>

The attention weights are then used to weigh the contributions of different nodes to the output. Let $A$ denote the attention weight matrix. The output of the GAT layer is computed as $H^{(k)} = \sigma (A H^{(k-1)} W)$ (the only difference between this equation and that of GCN is that the adjacent matrix $E$ is replaced with $A$).

In our implementation, `sparse_gat` and `sparse_softmax` are especially used to build the GAT layer. Simply put it, `sparse_gat` computes the $b_{ij}$ described above, and `sparse_softmax` is just a standard softmax over axis-1. `sparse_gat_grad_[sparse|dense]` only computes gradients for $H$ and $\vec{a}$, but not $E$. It makes sense because $E$ is an input graph property and doesn't require gradients.

We also implemented the multi-head attention structure in the paper in the `MultiHeadGAT` module. It takes the output of several GAT layers to stabilize the learning process. We only implemented the concatenation policy, which can be represented as

$$H^{(k)} = \parallel_{k=1}^K \sigma (A^{(k)} H^{(k-1)} W^{(k)})$$


## Models and Training

We implemented the Graph Convolution Model and the Graph Attention Model, together with a baseline fully-connected model, in `apps/gn.py`. The baseline model have roughly the same number of parameters as the Graph Convolution Model.

<p align="center">
  <img src="https://raw.github.com/zwang86/DLSys-Fall-2022-Report/main/img/graph_networks.png" alt="Graph Networks"/>
</p>


```
> python3 apps/gn.py [device] [model]
```

- device: cuda, cpu
- model: baseline, gc, gat

We evaluate the models on the Cora dataset. It a commonly used benchmark dataset for evaluating graph-based machine learning algorithms, on tasks such as node classification and link prediction. It consists of 2708 scientific publications classified into one of seven classes. The dataset includes the content (a feature vector) of the papers, the citation relationship among the papers, and the class labels for each paper. The dataset is already included in our repository under `data/`. You can also download it from https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz.

It is not trivial to split the train/validation/test sets on the graph data because data points are not independent.
We may apply graph networks in the *transductive* setting, which means that the sets are on the same graph.
The entire graph and all features (including validation/test features) are visible during training. Only validation/test labels are invisible.
There are many ways to handle this setting, and we choose to introduce a new network layer: `MaskedSoftmaxLoss`. It uses a mask so that only the train/validation output v.s. true labels are taken into loss computation.
Note that this is not the general case, and these sets may be on independent graphs in other applications of graph networks.


## Baseline Result

The output format for all the results is `train = {train accuracy}, {train loss}, val = {validation accuracy}, {validation loss}`.

In [ ]:
!python3 apps/gn.py cuda baseline

---Selecting backend:  cuda() ---
---Training baseline model---
epoch 1, elapsed 0.03s: train = 0.2950743494423792, 3.8048384189605713, val = 0.329136690647482, 3.6662778854370117
epoch 2, elapsed 0.04s: train = 0.29460966542750927, 2.0384480953216553, val = 0.3057553956834532, 2.073040008544922
epoch 3, elapsed 0.06s: train = 0.41821561338289964, 1.5862475633621216, val = 0.32194244604316546, 1.7222514152526855
epoch 4, elapsed 0.08s: train = 0.5455390334572491, 1.3163700103759766, val = 0.42805755395683454, 1.5298867225646973
epoch 5, elapsed 0.09s: train = 0.6979553903345725, 1.0869649648666382, val = 0.5413669064748201, 1.3500683307647705
epoch 6, elapsed 0.11s: train = 0.7727695167286245, 0.8974674940109253, val = 0.6205035971223022, 1.1971731185913086
epoch 7, elapsed 0.12s: train = 0.8336431226765799, 0.7194642424583435, val = 0.697841726618705, 1.0586415529251099
epoch 8, elapsed 0.14s: train = 0.8698884758364313, 0.5696262717247009, val = 0.7212230215827338, 0.9484922289848328

## Graph Network Result

In [ ]:
!python3 apps/gn.py cuda gc

---Selecting backend:  cuda() ---
---Training graph convolution network---
epoch 1, elapsed 0.02s: train = 0.29275092936802977, 6.315654754638672, val = 0.329136690647482, 5.342335224151611
epoch 2, elapsed 0.05s: train = 0.1570631970260223, 6.225630760192871, val = 0.1618705035971223, 6.348045825958252
epoch 3, elapsed 0.08s: train = 0.22072490706319703, 3.554532766342163, val = 0.21402877697841727, 3.7679293155670166
epoch 4, elapsed 0.11s: train = 0.2578996282527881, 2.47625994682312, val = 0.17805755395683454, 2.69366455078125
epoch 5, elapsed 0.13s: train = 0.266728624535316, 2.02108097076416, val = 0.210431654676259, 2.162968397140503
epoch 6, elapsed 0.16s: train = 0.5004646840148699, 1.476326823234558, val = 0.4658273381294964, 1.5470199584960938
epoch 7, elapsed 0.19s: train = 0.5013940520446096, 1.3853263854980469, val = 0.5251798561151079, 1.353360652923584
epoch 8, elapsed 0.21s: train = 0.5204460966542751, 1.3371285200119019, val = 0.5359712230215827, 1.3054759502410889
ep

In [ ]:
!python3 apps/gn.py cuda gat

---Selecting backend:  cuda() ---
---Training GAT model---
epoch 1, elapsed 0.13s: train = 0.0687732342007435, 2.616851806640625, val = 0.05575539568345324, 2.629812479019165
epoch 2, elapsed 0.25s: train = 0.34572490706319703, 2.2221007347106934, val = 0.4064748201438849, 2.1131505966186523
epoch 3, elapsed 0.36s: train = 0.27276951672862454, 2.2304086685180664, val = 0.22661870503597123, 2.3323240280151367
epoch 4, elapsed 0.45s: train = 0.3805762081784387, 1.6058175563812256, val = 0.3489208633093525, 1.6444553136825562
epoch 5, elapsed 0.55s: train = 0.6342936802973977, 1.3112070560455322, val = 0.6205035971223022, 1.3361002206802368
epoch 6, elapsed 0.63s: train = 0.675185873605948, 1.1696418523788452, val = 0.64568345323741, 1.2157604694366455
epoch 7, elapsed 0.72s: train = 0.7565055762081785, 1.011610507965088, val = 0.6996402877697842, 1.0769298076629639
epoch 8, elapsed 0.80s: train = 0.7973977695167286, 0.887193500995636, val = 0.7392086330935251, 0.973892092704773
epoch 9, 

It can be seen that both Graph Convolution and Graph Attention can achieve higher validation accuracy than the baseline model. All the training times are within few seconds. The training time of Graph Convolution is not too much more than the baseline model. Graph Attention is slower, but can achieve the highest accuracy.

# Side Goal: Optimize Speed with Computation Libraries

Note that the following results are produced on the personal computer of one of the team members (because we don't know how to install computation libraries on Google Colab). The CPU is Intel® Core™ i5-1135G7 @ 2.40GHz, which is not very powerful. There must be some number flucuations during the tests, but we believe the results are clear enough.

Out initial plan was to optimize with sparse-computation-related libraries (such as cuSPARSE). However, we did some profiling after finishing the project, and the results show that dense operations were still dominant in time in graph networks.

```
> python -m cProfile -s tottime apps/gn.py cpu gc
...
         890088 function calls (867987 primitive calls) in 64.627 seconds

   Ordered by: internal time
 
   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      720   62.374    0.087   62.374    0.087 {built-in method needle.backend_ndarray.ndarray_backend_cpu.matmul}
     1280    0.749    0.001    0.749    0.001 {built-in method needle.backend_ndarray.ndarray_backend_cpu.compact}
     2708    0.335    0.000    0.335    0.000 data.py:379(<listcomp>)
       40    0.138    0.003    0.139    0.003 {built-in method gc.collect}
      240    0.075    0.000    0.075    0.000 {built-in method needle.backend_ndarray.ndarray_backend_cpu.sparse_dense_matmul}
      120    0.070    0.001    0.070    0.001 {built-in method numpy.array}
      800    0.060    0.000    0.060    0.000 {built-in method needle.backend_ndarray.ndarray_backend_cpu.scalar_power}
     2360    0.057    0.000    0.057    0.000 {built-in method needle.backend_ndarray.ndarray_backend_cpu.ewise_add}
...
```

The reader may refer to the documentation of `cProfile` at https://docs.python.org/3/library/profile.html. We care about `tottime`, which means the total time spent on the function itself (not including other functions it calls).

It is clear that dense-dense matmul is the bottleneck. Based on the profiling results, we found that despite our initial plan to optimize with sparse-computation-related libraries, dense operations were still the dominant contributor to the computational time in our graph networks. This is not unexpected, as the number of floating-point operations (FLOPS) required for dense operations is higher than for sparse operations according to their definitions.

As a result, we chose to optimize dense-dense matmul with Intel MKL, a highly optimized computing library for Intel CPU. This is exposed as a `matmul_mkl` function in our CPU backend. This piece of code is only compiled when the compiler detects `mkl.h` available. The new profiling result with the optimization is:

```
> python -m cProfile -s tottime apps/gn.py cpu gc
...
         882248 function calls (860147 primitive calls) in 3.071 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1280    0.746    0.001    0.746    0.001 {built-in method needle.backend_ndarray.ndarray_backend_cpu.compact}
      720    0.740    0.001    0.740    0.001 {built-in method needle.backend_ndarray.ndarray_backend_cpu.matmul_mkl}
     2708    0.332    0.000    0.332    0.000 data.py:379(<listcomp>)
       40    0.151    0.004    0.151    0.004 {built-in method gc.collect}
      240    0.085    0.000    0.085    0.000 {built-in method needle.backend_ndarray.ndarray_backend_cpu.sparse_dense_matmul}
      120    0.070    0.001    0.070    0.001 {built-in method numpy.array}
      800    0.064    0.000    0.064    0.000 {built-in method needle.backend_ndarray.ndarray_backend_cpu.scalar_power}
     2360    0.064    0.000    0.064    0.000 {built-in method needle.backend_ndarray.ndarray_backend_cpu.ewise_add}
...
```

It is much (about 20x) faster, and the dense operations are still the bottleneck. So we gave up the idea of optimizing with sparse-computation-related libraries. We did similar profiling for the CUDA backend and GAT, which demonstrated similar results.